# industry & sector data retrieval

In [ ]:
# importing
import yfinance as yf
import pandas as pd
import numpy as np

#### in order to get maximize the number of matched sector and industry information, the ticker suffix .F will be used  in the following data retrieval procedure 

In [ ]:
# testing with XETRAS ticker suffix .DE for Deutsche Bank
deutsche = yf.Ticker("DBK.DE")
print(deutsche.info["sector"])
print(deutsche.info["industry"])

In [ ]:
# testing with XETRAS suffix .DE for Gamestop
gme = yf.Ticker("GS2C.DE")
print(gme.info['sector'])
print(gme.info['industry'])
# --> ImportError: html5lib not found, please install it

no sector and industry information for that ticker (see https://finance.yahoo.com/quote/GS2C.DE/profile?p=GS2C.DE)

however, using the .F suffix yields information on sector and insustry (see https://finance.yahoo.com/quote/GS2C.F?p=GS2C.F&.tsrc=fin-srch)

In [ ]:
# testing  with Frankfurt Stock Exchange suffix .F for Gamestop
gme = yf.Ticker("GS2C.F")
print(gme.info['sector'])
print(gme.info['industry'])

#### first, the unique tickers from the dbg-pds need to be extracted

In [ ]:
# extract tickers from full dbg-pds table
# use user-specific path of the data source 
df_full = pd.read_csv("../data/07_17_to_01_21_full.csv")

In [ ]:
# extract unique tickers from the Mnemonic column 
tickers = df_full["Mnemonic"].unique()

In [ ]:
# save unique tickers as CSV file
pd.DataFrame(tickers).to_csv("../data/unique_tickers.csv", header=None, index=None)

In [ ]:
# if already available, load unique tickers CSV
df_tickers = pd.read_csv("../data/unique_tickers.csv")

In [ ]:
# convert unique tickers to list
a = df_tickers.values.tolist()
tick_list = [x for [x] in a]

#### having the unique tickers in place, a function for the data collection needed to be formulated

In [ ]:
def get_ticker_sector_industry(ticker_list, progress=False):
    """
    returns a dataframe that maps tickers from a passed list to the respective sector and industry information
    using yfinance
    
    :param ticker_lsit: list of ticker symbols  
    :type ticker_list: list 
    :param progress: option to print progress information (default False)
    :type progress: bool
    """
    # creating a dataframe to store the information
    ticker_sector_df = pd.DataFrame(columns=["ticker", "sector", "industry"])
    # using Frankfurt Stock Exchange's .F ticker suffix
    suffix = ".F"
    # using NA for unavailable sector and industry information
    na = "NA"
    # loop trough list of tickers
    for tick in ticker_list:
        # only continue with the iteration if the ticker symbol is a string
        if isinstance(tick, str): 
            # option to display the progress of the dataframe creation
            if progress:
                print(f"{ticker_list.index(tick)+1} out of {len(ticker_list)}")
            # create a ticker that's readable for yfinance including the suffix
            ticker = yf.Ticker(tick + suffix)
            # circumvent errors for unavailable sector and industry information
            try:
                ticker_sector_df = ticker_sector_df.append({"ticker": tick,
                                                            "sector": ticker.info["sector"],
                                                            "industry": ticker.info["industry"]}, ignore_index=True)
            # in case of errors, fill in respective cells with NA
            except:            
                ticker_sector_df = ticker_sector_df.append({"ticker": tick,
                                                            "sector": na,
                                                            "industry": na}, ignore_index=True)
    return ticker_sector_df

In [ ]:
# testing with small hand-picked sample list
test_tickers = ["ADS", "BAS", "BAYN", "CBK", "EOAN"] 
print(get_ticker_sector_industry(test_tickers, progress=True))

In [ ]:
# testing with sample of the actual list
test_tick_list = tick_list[:20]
print(get_ticker_sector_industry(test_tick_list,progress=True))

In [ ]:
# create full mapping
df_ticker_industry_sector = get_ticker_sector_industry(tick_list)

In [ ]:
# inspect mapping
df_ticker_industry_sector.head()

In [ ]:
# save df_ticker_industry_sector as CSV file
pd.DataFrame(df_ticker_industry_sector).to_csv("../data/ticker_sector_industry_mapping.csv", index=None)

In [ ]:
# loading test
ticker_industy_sector_mapping = pd.read_csv(r"C:\Users\jacob\Google Drive\education\MADS @ FS\Y1Q3\visualising big data (vbd)\group project\data\ticker_sector_industry_mapping.csv")
# inspect mapping
ticker_industy_sector_mapping.head()